In [4]:
import numpy as np
import sys
import os
from sklearn.metrics import accuracy_score
import torch

sys.path.insert(1, os.getenv("NOVA_HOME"))
print(f"NOVA_HOME: {os.getenv('NOVA_HOME')}")

from src.common.utils import load_config_file
from src.datasets.dataset_config import DatasetConfig
from src.datasets.dataset_NOVA import DatasetNOVA
from src.models.architectures.NOVA_model import NOVAModel
from src.datasets.data_loader import get_dataloader


In [5]:
batch_size = 300
num_workers = 6

In [6]:
dataset_config_path = "./manuscript/dataset_config/OpenCellDatasetConfig"
dataset_config:DatasetConfig = load_config_file(dataset_config_path, 'dataset_config')

dataset = DatasetNOVA(dataset_config)

In [7]:
chkp_path = f"{os.getenv('NOVA_HOME')}/outputs/vit_models/pretrained_model/checkpoints/checkpoint_best.pth"
nova_model = NOVAModel.load_from_checkpoint(chkp_path)
nova_model

In [8]:
train_indexes, val_indexes, test_indexes = dataset.split()
dataloader_test = get_dataloader(dataset, batch_size, indexes=test_indexes, num_workers=num_workers, shuffle=False, drop_last=False)

/home/labs/hornsteinlab/sagyk/anaconda3/envs/nova/lib/python3.9/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [9]:
outputs, y_true = nova_model.infer(dataloader_test)
outputs = torch.from_numpy(outputs)
print(outputs.shape)

probs = torch.nn.functional.softmax(outputs, dim=1)
y_pred = dataset.id2label(torch.argmax(probs, dim=1))

accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy}")

/home/labs/hornsteinlab/sagyk/anaconda3/envs/nova/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1711403392949/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


torch.Size([170235, 1311])
Accuracy: 0.5942784973712809
